Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [5]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [58]:
KEYWORDS = ['обучение']

In [56]:
URL = 'https://habr.com/ru/all/'
params = {
'search_query': 'python'
}

res = requests.get(URL, params)
res.text

soup = BeautifulSoup(res.text, 'html.parser')
soup
posts = soup.find_all('article', class_='post')
posts


[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/NewTechAudit/" title="Автор публикации">
 <img class="user-info__image-pic user-info__image-pic_small" height="24" src="//habrastorage.org/getpro/habr/avatars/62e/cf1/3f1/62ecf13f1b5e83c0cced637ff786d5a1.png" width="24"/>
 <span class="user-info__nickname user-info__nickname_small">NewTechAudit</span>
 </a>
 <span class="post__time">сегодня в 09:07</span>
 </header>
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/525540/">Ловля цен. Практическое руководство в море закупок</a>
 </h2>
 <ul class="post__hubs inline-list">
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/hub/python/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 'Python'); }" rel="nofollow" title="Вы не подписаны на этот хаб">Python</

In [59]:
lst1=[]
lst2=[]
lst3=[]
for post in posts:
    post_id = post.parent.attrs.get('id')
    # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    prewie=post.find('div', class_='post__text')
    #print(prewie)
    prewie_lower = prewie.text.lower()
    if any([desired in prewie_lower for desired in KEYWORDS]):
        title_element = post.find('a', class_='post__title_link')
        time_block = post.find('span', class_='post__time')
        lst1.append(time_block.text)
        lst2.append(title_element.text)
        lst3.append(title_element.attrs.get('href'))
    continue
    hubs = post.find_all('a', class_='hub-link')
    for hub in hubs:
        hub_lower = hub.text.lower()
    # ищем вхождение хотя бы одного желаемого хаба
        if any([hub_lower in desired for desired in KEYWORDS]):
            title_element = post.find('a', class_='post__title_link')
            time_block = post.find('span', class_='post__time')
            lst1.append(time_block.text)
            lst2.append(title_element.text)
            lst3.append(title_element.attrs.get('href'))
    # так как пост уже нам подошел - дальше нет смысла проверять хабы
        continue
habr_posts = pd.DataFrame(list(zip(lst1, lst2, lst3)),columns =['Дата', 'Название', 'Ссылка'])
habr_posts

,Дата,Название,Ссылка
0,сегодня в 00:37,Мой опыт машинного обучения,https://habr.com/ru/post/525522/
1,вчера в 18:32,Насколько актуально машинное обучение в повсед...,https://habr.com/ru/post/525474/
